# This file is largely copied
see: https://www.kaggle.com/fabiendaniel/hyperparameter-tuning/notebook

We do need to change to make it our own.

# Team 3 - Merchant Category Recommendation
## Hyperparameter Tuning

### Team 3
- Vinicio De Sola
- Kevin Hanna
- Pri Nonis
- Bradley Nott


Here we add engineer new features and write the files out to:
    - input/engineered_train.pkl
    - input/engineered_test.pkl

In [1]:
import numpy               as np
import matplotlib.pyplot   as plt
import pandas              as pd
import seaborn             as sb

from datetime              import timedelta, datetime

from sklearn               import metrics
from sklearn.decomposition import PCA
from sklearn.cluster       import KMeans
from sklearn.mixture       import GaussianMixture
from sklearn.linear_model  import LinearRegression

from matplotlib.colors     import LogNorm

from IPython.display       import HTML, Markdown

from sklearn.metrics       import mean_squared_error

%matplotlib inline

np.random.seed(0)

import os
print(os.listdir("./input"))

['.gitignore', 'Data_Dictionary.xlsx', 'engineered_test.pkl', 'engineered_train.pkl', 'historical_transactions.csv', 'merchants.csv', 'new_merchant_transactions.csv', 'sample_submission.csv', 'test.csv', 'train.csv']


## Read in all the data files

In [58]:
train  = pd.read_pickle('input/engineered_train.pkl')
test   = pd.read_pickle('input/engineered_test.pkl' )

target = train['target']
del      train['target']

## A view of our training data dataframe

In [ ]:
display(train.head())
display(test.head())
display(target.head())

In [59]:
def compress(df) :
    pre = df.memory_usage().sum() / 1024**2 / 8
    cmp = {'f' : {np.finfo(np.float16).max : np.float16,
                  np.finfo(np.float32).max : np.float32,
                  np.finfo(np.float64).max : np.float64},
           'i' : {np.iinfo(np.int8   ).max : np.int8,
                  np.iinfo(np.int16  ).max : np.int16,
                  np.iinfo(np.int32  ).max : np.int32,
                  np.iinfo(np.int64  ).max : np.int64}}

    for c in df.columns :
        if  cmp.get(df[c].dtype.kind) :
            df[c] = df[c].astype(cmp[df[c].dtype.kind].get(min((n for n in cmp[df[c].dtype.kind].keys() if n > max(df[c].max(), abs(df[c].min()))))))

    end = df.memory_usage().sum() / 1024**2 / 8

    print(f'Memory Use Decreased to {end:5.2f} MB [{100 * (pre - end) / pre:.1f}% Reduction]')

    return df

In [61]:
train = compress(train)

for x in train.columns :
    if  mine[x] != train[x].dtype :
        print(f'{x:>35} : {orig[x]} to {train[x].dtype} != {mine[x]}')

features = train.columns
categorical_feats = [c for c in features if 'feature_' in c]


# target = reduce_mem_usage(target)
# features = reduce_mem_usage(features)
# categorical_feats = reduce_mem_usage(categorical_feats)
# categorical_feats = categorical_feats.astype('bool')

# display(target)

Mem. usage decreased to  2.82 Mb (-7.3% reduction)
                        feature_1_1 : uint8 to float16 != uint8
                        feature_1_2 : uint8 to float16 != uint8
                        feature_1_3 : uint8 to float16 != uint8
                        feature_1_4 : uint8 to float16 != uint8
                        feature_1_5 : uint8 to float16 != uint8
                        feature_2_1 : uint8 to float16 != uint8
                        feature_2_2 : uint8 to float16 != uint8
                        feature_2_3 : uint8 to float16 != uint8


In [ ]:
def LGB_CV(
          max_depth,
          num_leaves,
          min_data_in_leaf,
          feature_fraction,
          bagging_fraction,
          lambda_l1
         ):
    
    ### Delete this
   
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    oof = np.zeros(train.shape[0])

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
        print("fold n°{}".format(fold_))
        trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                               label=target.iloc[trn_idx],
                               categorical_feature=categorical_feats)
        val_data = lgb.Dataset(train.iloc[val_idx][features],
                               label=target.iloc[val_idx],
                               categorical_feature=categorical_feats)
    
        param = {
            'num_leaves': int(num_leaves),
            'min_data_in_leaf': int(min_data_in_leaf), 
            'objective':'regression',
            'max_depth': int(max_depth),
            'learning_rate': 0.01,
            "boosting": "gbdt",
            "feature_fraction": feature_fraction,
            "bagging_freq": 1,
            "bagging_fraction": bagging_fraction ,
            "bagging_seed": 11,
            "metric": 'rmse',
            "lambda_l1": lambda_l1,
            "verbosity": -1
        }
    
        clf = lgb.train(param,
                        trn_data,
                        10000,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=500,
                        early_stopping_rounds = 200)
        
        oof[val_idx] = clf.predict(train.iloc[val_idx][features],
                                   num_iteration=clf.best_iteration)
        
        del clf, trn_idx, val_idx
        gc.collect()
        
    return -mean_squared_error(oof, target)**0.5

In [ ]:
from bayes_opt import BayesianOptimization
LGB_BO = BayesianOptimization(LGB_CV, {
    'max_depth': (4, 10),
    'num_leaves': (5, 130),
    'min_data_in_leaf': (10, 150),
    'feature_fraction': (0.7, 1.0),
    'bagging_fraction': (0.7, 1.0),
    'lambda_l1': (0, 6)
    })

In [ ]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod(
            (datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))



In [ ]:
import lightgbm as lgb
import gc
from sklearn.model_selection import KFold
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print('-'*126)

start_time = timer(None)
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    LGB_BO.maximize(init_points=2, n_iter=20, acq='ei', xi=0.0)
timer(start_time)